In [2]:
from utilities import load_jsonl
data = load_jsonl("../input/raw/chatGPT.jsonl")

# build wiki sentence jsonl - each json will have:
* "sid"
* "pid"
* "sentence_num"
* "has_error"
* "num_facts"
* "has_annotations"
* "decon_sentence"
* "title"

In [22]:
"""
level 1 keys: sentence for which there are annotations
level 2 keys: 'has_error':
              'facts':
              'num_facts':
              'has_facts'

"""
annotated_sentences_dict = {}

for i in range(len(data)):
    topic = data[i]['topic']
    if data[i]["annotations"]:
        for obj1 in data[i]["annotations"]:
            sentence = obj1['text']
            data_dict = {}
            facts = []
            labels = []
            has_facts = False
            if obj1['human-atomic-facts']:
                has_facts = True
                for obj2 in obj1['human-atomic-facts']:
                    facts.append(obj2['text'])
                    labels.append(obj2['label'])
            data_dict['facts'] = facts
            data_dict['num_facts'] = len(facts)
            data_dict['has_error'] = 'NS' in labels
            data_dict['has_facts'] = has_facts
            data_dict['topic'] = topic
            annotated_sentences_dict[sentence] = data_dict
            
    

        
        
    

In [23]:
len(annotated_sentences_dict)

1241

# create jsonl with desired structure

In [24]:
wiki_jsonl = []
sid = 0
for pid in range(len(decon_sents)):
    for sentence_num, decon_sentence in enumerate(decon_sents[pid]["decon_sentences"]):
        data_dict = {}
        data_dict['pid'] = pid
        data_dict['sid'] = sid
        data_dict['decon_sentence'] = decon_sentence
        sentence = decon_sents[pid]["sentences"][sentence_num]
        data_dict['sentence'] = sentence
        data_dict['sentence_num'] = sentence_num
        data_dict['has_error'] = False
        data_dict['has_facts'] = False
        data_dict['facts'] = []
        data_dict['num_facts'] = 0
        data_dict['has_annotations'] = False
        if sentence in annotated_sentences_dict:
            data_dict['has_error'] = annotated_sentences_dict[sentence]['has_error']
            data_dict['has_facts'] = annotated_sentences_dict[sentence]['has_facts']
            data_dict['facts'] = annotated_sentences_dict[sentence]['facts']
            data_dict['num_facts'] = annotated_sentences_dict[sentence]['num_facts']
            data_dict['has_annotations'] = True
            data_dict['topic'] = annotated_sentences_dict[sentence]['topic']
            data_dict['fact_queries'] = [f"{annotated_sentences_dict[sentence]['topic']}:{fact}" for fact in data_dict['facts']]
    
        wiki_jsonl.append(data_dict)
        sid += 1
    

In [33]:
import json
# Write to a .jsonl file
with open("../input/processed/wiki_input_processed.jsonl", 'w') as f:
    for entry in wiki_jsonl:
        f.write(json.dumps(entry) + '\n')